In [2]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
from pylab import *

In [3]:
# Define constants
AFM_FOLDER = '../data_vault/AFM/'
PNG_FOLDER = '../data_vault/AFM/png/'
DIM = 512

In [4]:
# Read an AFM file. Return image data, pixel size, and units
def read_file(file):
    f = open(AFM_FOLDER + file, encoding = 'latin-1')
    
    header = True
    hv = 0
    bpp = 0
    imarray = []
    units = ''
    for line in f:
        cols = line.split()
        
        if len(cols) > 1:
            if cols[0] == '\Bytes/pixel:' and bpp == 0: bpp = int(cols[1])
            if cols[0] + cols[1] == '\Scansize:' and units == '': 
                size = float(cols[2])
                units = cols[3]
            if cols[0] + cols[1] == '\@2:Zscale:' and hv == 0: hv = float(cols[-2])
            if cols[0] == '\Samps/line:': samps_per_line = int(cols[1])
        
        if not header and len(imarray) < 1024 and len(cols) > 1:
            scale = hv/(2**(8*bpp))**2/2*25e5 # in nm
            imarray.append([float(i)*scale for i in cols]) #image stored as rows, columns (upright)
            
        if line.strip() == "": 
            header = False
            
    # print('Scale: ' + str(scale) + ' nm')     
    imarraytemp = array(imarray[0:DIM-1])
    
    # find first nonzero row
    start=0
    while sum(imarraytemp[start,:])==0:
        start+=1
    end=len(imarraytemp)-1
    while sum(imarraytemp[end,:])==0:
        end-=1
    imarray=imarraytemp[start:end+1,:]
    f.close()
    
    return imarray, size / samps_per_line, units

In [5]:
def plot_image(imarray, pixel_size, units, file, save = False):

    fig=figure(figsize = (7,7))
    im = imshow(imarray, extent = (0,imarray.shape[1]*pixel_size,imarray.shape[0]*pixel_size,0), 
                cmap = get_cmap('afmhot'))

    xlabel('Position (' + units + ')', fontsize = 14)
    divider = make_axes_locatable(gca())
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = colorbar(im, cax=cax)
    clim(-5,5)
    ylabel('Height (nm)', fontsize = 14)   
    
    if save:
        savefig(PNG_FOLDER + str(file) + '.png',
            bbox_inches = 'tight', pad_inches= 0.2)

In [6]:
def ascii_to_image(file, save = False):
    imarray, pixel_size, units=read_file(file)
    plot_image(imarray, pixel_size, units, file, save)

In [3]:
def convert_all_ascii():
    for file in os.listdir(AFM_FOLDER):
        if '.' in file and file[0] != '.' and not (file + '.png') in os.listdir(PNG_FOLDER):
            print(file)
            ascii_to_image(file, save = True)
    close()